<a href="https://colab.research.google.com/github/Manos-Mak120/-/blob/Articles-Scrapper/NEWSBOMB_BIDEN_ARTICLES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

from tqdm import tqdm

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')
def get_article_elements_from_soup(soup, article_url):
    # title
    try:
        title = soup.find("h1").text.strip()
    except:
        title = ""

    # datetime
    try:
        datetime_str = soup.find("time")["datetime"]
        date_str = datetime_str.split("T")[0]
        time_str = datetime_str.split("T")[1]
    except:
        date_str = ""
        time_str = ""
    # section
    try:
        section_div = soup.find("div", class_="main-category-block") or soup.find("div", class_="main-category")
        section = section_div.text.strip() if section_div else ""
    except:
        section = ""

    # author
    try:
      author_tag = soup.find("a", class_="author-name") or soup.find("span", class_="author-name")
      author = author_tag.text.strip() if author_tag else ""
    except:
        author = ""
    # excerpt
    try:
        excerpt_div = soup.find("div", class_="p.main-intro") or soup.find(itemprop="description") or soup.find("div", class_="nx-excerpt")
        excerpt = excerpt_div.text.strip() if excerpt_div else ""
    except:
        excerpt = ""

    # text body
    try:
      content_div = soup.find("div", class_="main-text story-fulltext")
      elements = content_div.find_all(["p", "h2", "h3", "ul", "ol", "li"]) if content_div else []
      text_body = " ".join(el.get_text(strip=True) for el in elements if el.get_text(strip=True))
    except:
        text_body = ""

    # feature image
    try:
        img_block = soup.find("div", class_="article-image") or soup.find("figure")
        feat_img_link = img_block.find("img")["src"] if img_block and img_block.find("img") else ""
    except:
        feat_img_link = ""

    # image caption
    try:
        caption_div = soup.find("div", class_="image-caption") or soup.find("figcaption")
        image_caption = caption_div.text.strip() if caption_div else ""
    except:
        image_caption = ""

    articleD = {
        "article_url": article_url,
        "title": title,
        "date": date_str,
        "time": time_str,
        "section": section,
        "author": author,
        "excerpt": excerpt,
        "text_body": text_body,
        "feat_img_link": feat_img_link,
        "feat_img_cation": image_caption
    }

    return articleD

In [ ]:
def scrape_newsbomb_articles_pages(teasers_df):
  all_articles_dataL = []
  base_url = "https://www.newsbomb.gr"
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):
    article_url_relative = row['article_url']
    # Check if the article_url_relative is already a full URL
    if article_url_relative.startswith(base_url):
        article_url = article_url_relative
    else:
        article_url = base_url + article_url_relative
    soup = get_webpage_soup(article_url)
    if soup:
     articleD = get_article_elements_from_soup(soup, article_url)
     all_articles_dataL.append(articleD)
    delay = random.uniform(1, 3)
    time.sleep(delay)
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
biden_teasers_df = pd.read_csv("/content/drive/MyDrive/Εργασία Python/scrappers/BIDEN(21)-SCRAPER.CSV")
biden_teasers_df.head()

,title,article_url,date,time,section,excerpt
0,Μπάιντεν: Διάγγελμα για το Αφγανιστάν- O Αμερι...,https://www.newsbomb.gr/kosmos/story/1228361/m...,2021-08-31,22:55,ΚΟΣΜΟΣ,Mήνυμα στον αμερικανικό λαό απηύθυνε ο πρόεδρο...
1,Μπλίνκεν: Θα συνεργαστούμε με τους Ταλιμπάν εά...,https://www.newsbomb.gr/kosmos/story/1228111/m...,2021-08-31,06:15,ΚΟΣΜΟΣ,«Κάθε βήμα που θα κάνουμε δεν θα βασίζεται στο...
2,Νέο διάγγελμα Μπάιντεν για το Αφγανιστάν,https://www.newsbomb.gr/kosmos/story/1228104/n...,2021-08-31,02:00,ΚΟΣΜΟΣ,Ο αμερικανός πρόεδρος Τζο Μπάιντεν θα εκφραστε...
3,Τυφώνας Άιντα: Η στιγμή που οι σφοδροί άνεμοι ...,https://www.newsbomb.gr/kosmos/story/1227974/t...,2021-08-30,15:17,ΚΟΣΜΟΣ,Ο τυφώνας Άιντα έφθασε πάνω από το έδαφος όντα...
4,Σάλος με τον Μπαίντεν να κοιτά το ρολόι του στ...,https://www.newsbomb.gr/kosmos/story/1227963/s...,2021-08-30,14:45,ΚΟΣΜΟΣ,Πύρα δέχεται ο πρόεδρος των ΗΠΑ Τζο Μπάιντεν μ...


In [ ]:
biden_teasers_df_test = biden_teasers_df.head(10)
biden_teasers_df_test

,title,article_url,date,time,section,excerpt
0,Μπάιντεν: Διάγγελμα για το Αφγανιστάν- O Αμερι...,https://www.newsbomb.gr/kosmos/story/1228361/m...,2021-08-31,22:55,ΚΟΣΜΟΣ,Mήνυμα στον αμερικανικό λαό απηύθυνε ο πρόεδρο...
1,Μπλίνκεν: Θα συνεργαστούμε με τους Ταλιμπάν εά...,https://www.newsbomb.gr/kosmos/story/1228111/m...,2021-08-31,06:15,ΚΟΣΜΟΣ,«Κάθε βήμα που θα κάνουμε δεν θα βασίζεται στο...
2,Νέο διάγγελμα Μπάιντεν για το Αφγανιστάν,https://www.newsbomb.gr/kosmos/story/1228104/n...,2021-08-31,02:00,ΚΟΣΜΟΣ,Ο αμερικανός πρόεδρος Τζο Μπάιντεν θα εκφραστε...
3,Τυφώνας Άιντα: Η στιγμή που οι σφοδροί άνεμοι ...,https://www.newsbomb.gr/kosmos/story/1227974/t...,2021-08-30,15:17,ΚΟΣΜΟΣ,Ο τυφώνας Άιντα έφθασε πάνω από το έδαφος όντα...
4,Σάλος με τον Μπαίντεν να κοιτά το ρολόι του στ...,https://www.newsbomb.gr/kosmos/story/1227963/s...,2021-08-30,14:45,ΚΟΣΜΟΣ,Πύρα δέχεται ο πρόεδρος των ΗΠΑ Τζο Μπάιντεν μ...
5,Αφγανιστάν: Ξέσπασε η μητέρα πεζοναύτη που σκο...,https://www.newsbomb.gr/kosmos/story/1227881/a...,2021-08-30,10:07,ΚΟΣΜΟΣ,Αφγανιστάν: Ξέσπασε κατά του Αμερικανού προέδρ...
6,Τυφώνας Άιντα: Σε κατάσταση καταστροφής κήρυξε...,https://www.newsbomb.gr/kosmos/story/1227853/t...,2021-08-30,08:42,ΚΟΣΜΟΣ,Τυφώνας Άιντα: Ο Άιντα έφθασε πάνω από το έδαφ...
7,ΗΠΑ: Θλιμμένος ο Μπάιντεν μπροστά στη θέα των ...,https://www.newsbomb.gr/kosmos/story/1227787/h...,2021-08-29,22:21,ΚΟΣΜΟΣ,Έκδηλη ήταν η συναισθηματική φόρτιση του Τζο Μ...
8,ΗΠΑ: Ο Tzo Μπάιντεν θα υποδεχτεί τα φέρετρα τω...,https://www.newsbomb.gr/kosmos/story/1227729/h...,2021-08-29,16:35,ΚΟΣΜΟΣ,Αφγανιστάν - Όλες οι ειδήσεις: Ο Αμερικανός πρ...
9,«Κόκκινος συναγερμός» από Μπάιντεν για Αφγανισ...,https://www.newsbomb.gr/kosmos/story/1227605/k...,2021-08-28,22:32,ΚΟΣΜΟΣ,"Ο πρόεδρος των ΗΠΑ, Τζο Μπάιντεν, εκτιμά ότι ε..."


In [ ]:
articles_df = scrape_newsbomb_articles_pages(biden_teasers_df)
articles_df

Processing rows: 100%|██████████| 911/911 [45:34<00:00,  3.00s/it]



Scraping completed. Fetshed 911 articles.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
0,https://www.newsbomb.gr/kosmos/story/1228361/m...,Μπάιντεν: Διάγγελμα για το Αφγανιστάν- O Αμερι...,2021-08-31,22:55:19+03:00,ΚΟΣΜΟΣ,Νατάσα Παυλοπούλου,Mήνυμα στον αμερικανικό λαό απηύθυνε ο πρόεδρο...,"Μετά από δύο εβδομάδες πτήσεων εκκένωσης, μια ...",https://img.bbmd.gr/img/1260/max/82/2021/08/31...,
1,https://www.newsbomb.gr/kosmos/story/1228111/m...,Μπλίνκεν: Θα συνεργαστούμε με τους Ταλιμπάν εά...,2021-08-31,06:15:09+03:00,ΚΟΣΜΟΣ,Γιάννης Καλύβας,«Κάθε βήμα που θα κάνουμε δεν θα βασίζεται στο...,Οι ΗΠΑ είναι διατεθειμένες να «συνεργαστούν» μ...,https://img.bbmd.gr/img/1260/max/82/2021/08/31...,(Jonathan Ernst/Pool via AP)
2,https://www.newsbomb.gr/kosmos/story/1228104/n...,Νέο διάγγελμα Μπάιντεν για το Αφγανιστάν,2021-08-31,02:00:50+03:00,ΚΟΣΜΟΣ,Γιάννης Καλύβας,Ο αμερικανός πρόεδρος Τζο Μπάιντεν θα εκφραστε...,"Όπως διευκρίνισε η αμερικανική προεδρία, ο κ. ...",https://img.bbmd.gr/img/1260/max/82/2021/08/30...,(AP Photo/Manuel Balce Ceneta)
3,https://www.newsbomb.gr/kosmos/story/1227974/t...,Τυφώνας Άιντα: Η στιγμή που οι σφοδροί άνεμοι ...,2021-08-30,15:17:36+03:00,ΚΟΣΜΟΣ,Newsbomb,Ο τυφώνας Άιντα έφθασε πάνω από το έδαφος όντα...,Viral έχει γίνει στο διαδίκτυο ένα σοκαριστικό...,https://img.bbmd.gr/img/1260/max/82/2021/08/30...,Οι δυνατοί άνεμοι ξηλώνουν την οροφή του νοσοκ...
4,https://www.newsbomb.gr/kosmos/story/1227963/s...,Σάλος με τον Μπαίντεν να κοιτά το ρολόι του στ...,2021-08-30,14:45:20+03:00,ΚΟΣΜΟΣ,Newsbomb,Πύρα δέχεται ο πρόεδρος των ΗΠΑ Τζο Μπάιντεν μ...,"Σύμφωνα με την Daily Mail, o Τζο Μπάιντεν ταξί...",https://img.bbmd.gr/img/1260/max/82/2021/08/30...,"Κατά τη διάρκεια της τελετής, ο Μπάιντεν εμφαν..."
...,...,...,...,...,...,...,...,...,...,...
906,https://www.newsbomb.gr/kosmos/story/1121995/e...,Εκλογές ΗΠΑ 2020: Όλα τα μάτια στραμμένα στο π...,2020-09-28,19:22:55+03:00,ΚΟΣΜΟΣ,Newsbomb,Σε λίγες ώρες (4 τα ξημερώματα ώρα Ελλάδος) θα...,Η πανδημία του κορονοϊού άλλαξε ριζικά τον τρό...,https://img.bbmd.gr/img/1260/max/82/2020/09/28...,
907,https://www.newsbomb.gr/kosmos/story/1121668/a...,Αμερικανικές εκλογές 2020: «Θρίλερ» στις κρίσι...,2020-09-27,19:45:14+03:00,ΚΟΣΜΟΣ,Newsbomb,"Το προβάδισμα του Τζο Μπάιντεν, του υποψηφίου ...","Σύμφωνα με τις τελευταίες δημοσκοπήσεις, η μάχ...",https://img.bbmd.gr/img/1260/max/82/2020/09/27...,
908,https://www.newsbomb.gr/kosmos/story/1117092/t...,Τραμπ: Ο Τζο Μπάιντεν ντοπάρεται για να δίνει ...,2020-09-12,06:15:23+03:00,ΚΟΣΜΟΣ,Newsbomb,"Ο πρόεδρος των ΗΠΑ Ντόναλντ Τραμπ, χωρίς ακόμη...",«Νομίζω ότι ενδεχομένως εμπλέκονται πιθανώς ου...,https://img.bbmd.gr/img/1260/max/82/2020/09/12...,(AP Photo/Andrew Harnik)
909,https://www.newsbomb.gr/kosmos/story/1117065/o...,Ο πρόεδρος Τραμπ και ο πρώην αντιπρόεδρος Τζο ...,2020-09-11,23:57:48+03:00,ΚΟΣΜΟΣ,Newsbomb,Ο υποψήφιος των Δημοκρατικών για την προεδρία ...,Μπάιντεν και Πενς χαιρετίστηκαν «αγγίζοντας» τ...,https://img.bbmd.gr/img/1260/max/82/2020/09/11...,AP PHOTOS


In [ ]:
articles_df['text_body'][0]  #έλεγχος ότι πήρε όλο το κείμενο

'Μετά από δύο εβδομάδες πτήσεων εκκένωσης, μια τιτάνια προσπάθεια που αμαυρώθηκε απο μίαβομβιστική επίθεση αυτοκτονίας που σκότωσε 13 Αμερικανούς στρατιώτες και δεκάδες Αφγανούς, ο Μπάιντεν άφησε τους επικεφαλής του Πενταγώνου και του Στέιτ Ντιπάρτμεντ να κάνουν την τελική ανακοίνωση αποχώρησης τη Δευτέρα. Με τη σημερινή ομιλία του από το Λευκό Οίκο, ο Αμερικανός πρόεδρος που έχει βρεθεί στο στόχαστρο για τα λάθη και τις αστοχίες του στο Αφγανιστάν, επιχείρησε να εξηγήσειτο όραμά του. Οι Ρεπουμπλικανοί, με επικεφαλής τον προκάτοχό του, Ντόναλντ Τραμπ,περιγράφουν την έξοδο ως μια ταπεινωτική αποτυχία, μια ήττα που ξεπερνά ακόμη και τη φυγή των Αμερικανών από τη Σαϊγκόν το 1975, και ένα μήνυμα ότι οι Ηνωμένες Πολιτείες...εγκαταλείπουν Ο Μπάιντεν εξέφρασε εναεντελώς διαφορετικό επιχείρημα:ότι η διαφυγή από έναν πόλεμο στην άλλη άκρη του κόσμου δεν θα ήταν ποτέ ένα εύκολο εγχείρημα και ότι είναι ο πρώτος πρόεδρος εδώ και 20 χρόνια πουείχε το θάρρος να ολοκληρώσει αυτό που όλοι ήξεραν ότι έ

In [ ]:
articles_df['feat_img_link'][0]

'https://img.bbmd.gr/img/1260/max/82/2021/08/31/AP1203210889121.jpg?t=BTTsRBikfakR-icSgRSAHQ'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
biden_teasers_df = pd.read_csv("/content/drive/MyDrive/Εργασία Python/scrappers/BIDEN(21)-SCRAPER.CSV")
biden_teasers_df.head()

,title,article_url,date,time,section,excerpt
0,Μπάιντεν: Διάγγελμα για το Αφγανιστάν- O Αμερι...,https://www.newsbomb.gr/kosmos/story/1228361/m...,2021-08-31,22:55,ΚΟΣΜΟΣ,Mήνυμα στον αμερικανικό λαό απηύθυνε ο πρόεδρο...
1,Μπλίνκεν: Θα συνεργαστούμε με τους Ταλιμπάν εά...,https://www.newsbomb.gr/kosmos/story/1228111/m...,2021-08-31,06:15,ΚΟΣΜΟΣ,«Κάθε βήμα που θα κάνουμε δεν θα βασίζεται στο...
2,Νέο διάγγελμα Μπάιντεν για το Αφγανιστάν,https://www.newsbomb.gr/kosmos/story/1228104/n...,2021-08-31,02:00,ΚΟΣΜΟΣ,Ο αμερικανός πρόεδρος Τζο Μπάιντεν θα εκφραστε...
3,Τυφώνας Άιντα: Η στιγμή που οι σφοδροί άνεμοι ...,https://www.newsbomb.gr/kosmos/story/1227974/t...,2021-08-30,15:17,ΚΟΣΜΟΣ,Ο τυφώνας Άιντα έφθασε πάνω από το έδαφος όντα...
4,Σάλος με τον Μπαίντεν να κοιτά το ρολόι του στ...,https://www.newsbomb.gr/kosmos/story/1227963/s...,2021-08-30,14:45,ΚΟΣΜΟΣ,Πύρα δέχεται ο πρόεδρος των ΗΠΑ Τζο Μπάιντεν μ...


In [ ]:
biden_teasers_df_test = biden_teasers_df.head(10)
biden_teasers_df_test

,title,article_url,date,time,section,excerpt
0,Μπάιντεν: Διάγγελμα για το Αφγανιστάν- O Αμερι...,https://www.newsbomb.gr/kosmos/story/1228361/m...,2021-08-31,22:55,ΚΟΣΜΟΣ,Mήνυμα στον αμερικανικό λαό απηύθυνε ο πρόεδρο...
1,Μπλίνκεν: Θα συνεργαστούμε με τους Ταλιμπάν εά...,https://www.newsbomb.gr/kosmos/story/1228111/m...,2021-08-31,06:15,ΚΟΣΜΟΣ,«Κάθε βήμα που θα κάνουμε δεν θα βασίζεται στο...
2,Νέο διάγγελμα Μπάιντεν για το Αφγανιστάν,https://www.newsbomb.gr/kosmos/story/1228104/n...,2021-08-31,02:00,ΚΟΣΜΟΣ,Ο αμερικανός πρόεδρος Τζο Μπάιντεν θα εκφραστε...
3,Τυφώνας Άιντα: Η στιγμή που οι σφοδροί άνεμοι ...,https://www.newsbomb.gr/kosmos/story/1227974/t...,2021-08-30,15:17,ΚΟΣΜΟΣ,Ο τυφώνας Άιντα έφθασε πάνω από το έδαφος όντα...
4,Σάλος με τον Μπαίντεν να κοιτά το ρολόι του στ...,https://www.newsbomb.gr/kosmos/story/1227963/s...,2021-08-30,14:45,ΚΟΣΜΟΣ,Πύρα δέχεται ο πρόεδρος των ΗΠΑ Τζο Μπάιντεν μ...
5,Αφγανιστάν: Ξέσπασε η μητέρα πεζοναύτη που σκο...,https://www.newsbomb.gr/kosmos/story/1227881/a...,2021-08-30,10:07,ΚΟΣΜΟΣ,Αφγανιστάν: Ξέσπασε κατά του Αμερικανού προέδρ...
6,Τυφώνας Άιντα: Σε κατάσταση καταστροφής κήρυξε...,https://www.newsbomb.gr/kosmos/story/1227853/t...,2021-08-30,08:42,ΚΟΣΜΟΣ,Τυφώνας Άιντα: Ο Άιντα έφθασε πάνω από το έδαφ...
7,ΗΠΑ: Θλιμμένος ο Μπάιντεν μπροστά στη θέα των ...,https://www.newsbomb.gr/kosmos/story/1227787/h...,2021-08-29,22:21,ΚΟΣΜΟΣ,Έκδηλη ήταν η συναισθηματική φόρτιση του Τζο Μ...
8,ΗΠΑ: Ο Tzo Μπάιντεν θα υποδεχτεί τα φέρετρα τω...,https://www.newsbomb.gr/kosmos/story/1227729/h...,2021-08-29,16:35,ΚΟΣΜΟΣ,Αφγανιστάν - Όλες οι ειδήσεις: Ο Αμερικανός πρ...
9,«Κόκκινος συναγερμός» από Μπάιντεν για Αφγανισ...,https://www.newsbomb.gr/kosmos/story/1227605/k...,2021-08-28,22:32,ΚΟΣΜΟΣ,"Ο πρόεδρος των ΗΠΑ, Τζο Μπάιντεν, εκτιμά ότι ε..."


In [ ]:
articles_df = scrape_newsbomb_articles_pages(biden_teasers_df)
articles_df

Processing rows: 100%|██████████| 911/911 [44:37<00:00,  2.94s/it]



Scraping completed. Fetshed 911 articles.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
0,https://www.newsbomb.gr/kosmos/story/1228361/m...,Μπάιντεν: Διάγγελμα για το Αφγανιστάν- O Αμερι...,2021-08-31,22:55:19+03:00,ΚΟΣΜΟΣ,Νατάσα Παυλοπούλου,Mήνυμα στον αμερικανικό λαό απηύθυνε ο πρόεδρο...,"Μετά από δύο εβδομάδες πτήσεων εκκένωσης, μια ...",https://img.bbmd.gr/img/1260/max/82/2021/08/31...,
1,https://www.newsbomb.gr/kosmos/story/1228111/m...,Μπλίνκεν: Θα συνεργαστούμε με τους Ταλιμπάν εά...,2021-08-31,06:15:09+03:00,ΚΟΣΜΟΣ,Γιάννης Καλύβας,«Κάθε βήμα που θα κάνουμε δεν θα βασίζεται στο...,Οι ΗΠΑ είναι διατεθειμένες να «συνεργαστούν» μ...,https://img.bbmd.gr/img/1260/max/82/2021/08/31...,(Jonathan Ernst/Pool via AP)
2,https://www.newsbomb.gr/kosmos/story/1228104/n...,Νέο διάγγελμα Μπάιντεν για το Αφγανιστάν,2021-08-31,02:00:50+03:00,ΚΟΣΜΟΣ,Γιάννης Καλύβας,Ο αμερικανός πρόεδρος Τζο Μπάιντεν θα εκφραστε...,"Όπως διευκρίνισε η αμερικανική προεδρία, ο κ. ...",https://img.bbmd.gr/img/1260/max/82/2021/08/30...,(AP Photo/Manuel Balce Ceneta)
3,https://www.newsbomb.gr/kosmos/story/1227974/t...,Τυφώνας Άιντα: Η στιγμή που οι σφοδροί άνεμοι ...,2021-08-30,15:17:36+03:00,ΚΟΣΜΟΣ,Newsbomb,Ο τυφώνας Άιντα έφθασε πάνω από το έδαφος όντα...,Viral έχει γίνει στο διαδίκτυο ένα σοκαριστικό...,https://img.bbmd.gr/img/1260/max/82/2021/08/30...,Οι δυνατοί άνεμοι ξηλώνουν την οροφή του νοσοκ...
4,https://www.newsbomb.gr/kosmos/story/1227963/s...,Σάλος με τον Μπαίντεν να κοιτά το ρολόι του στ...,2021-08-30,14:45:20+03:00,ΚΟΣΜΟΣ,Newsbomb,Πύρα δέχεται ο πρόεδρος των ΗΠΑ Τζο Μπάιντεν μ...,"Σύμφωνα με την Daily Mail, o Τζο Μπάιντεν ταξί...",https://img.bbmd.gr/img/1260/max/82/2021/08/30...,"Κατά τη διάρκεια της τελετής, ο Μπάιντεν εμφαν..."
...,...,...,...,...,...,...,...,...,...,...
906,https://www.newsbomb.gr/kosmos/story/1121995/e...,Εκλογές ΗΠΑ 2020: Όλα τα μάτια στραμμένα στο π...,2020-09-28,19:22:55+03:00,ΚΟΣΜΟΣ,Newsbomb,Σε λίγες ώρες (4 τα ξημερώματα ώρα Ελλάδος) θα...,Η πανδημία του κορονοϊού άλλαξε ριζικά τον τρό...,https://img.bbmd.gr/img/1260/max/82/2020/09/28...,
907,https://www.newsbomb.gr/kosmos/story/1121668/a...,Αμερικανικές εκλογές 2020: «Θρίλερ» στις κρίσι...,2020-09-27,19:45:14+03:00,ΚΟΣΜΟΣ,Newsbomb,"Το προβάδισμα του Τζο Μπάιντεν, του υποψηφίου ...","Σύμφωνα με τις τελευταίες δημοσκοπήσεις, η μάχ...",https://img.bbmd.gr/img/1260/max/82/2020/09/27...,
908,https://www.newsbomb.gr/kosmos/story/1117092/t...,Τραμπ: Ο Τζο Μπάιντεν ντοπάρεται για να δίνει ...,2020-09-12,06:15:23+03:00,ΚΟΣΜΟΣ,Newsbomb,"Ο πρόεδρος των ΗΠΑ Ντόναλντ Τραμπ, χωρίς ακόμη...",«Νομίζω ότι ενδεχομένως εμπλέκονται πιθανώς ου...,https://img.bbmd.gr/img/1260/max/82/2020/09/12...,(AP Photo/Andrew Harnik)
909,https://www.newsbomb.gr/kosmos/story/1117065/o...,Ο πρόεδρος Τραμπ και ο πρώην αντιπρόεδρος Τζο ...,2020-09-11,23:57:48+03:00,ΚΟΣΜΟΣ,Newsbomb,Ο υποψήφιος των Δημοκρατικών για την προεδρία ...,Μπάιντεν και Πενς χαιρετίστηκαν «αγγίζοντας» τ...,https://img.bbmd.gr/img/1260/max/82/2020/09/11...,AP PHOTOS


In [ ]:
from datetime import datetime

start_date = datetime(2020, 11, 20)
end_date = datetime(2021, 4, 30)

articles_df['date'] = pd.to_datetime(articles_df['date'], errors='coerce')

filtered_articles_df = articles_df[(articles_df['date'] >= start_date) & (articles_df['date'] <= end_date)].copy()

print(f"\nFiltered articles from 20/11/2020 to April 2021: {len(filtered_articles_df)}")


Filtered articles from 20/11/2020 to April 2021: 369


In [ ]:
duplicates_url = filtered_articles_df[filtered_articles_df.duplicated(subset=['article_url'], keep=False)]
print(f"\nArticles with duplicate URLs: {len(duplicates_url)}")
if not duplicates_url.empty:
    print(duplicates_url[['article_url', 'title']].sort_values(by='article_url'))

# Check for articles without title
no_title = filtered_articles_df[filtered_articles_df['title'].isnull() | (filtered_articles_df['title'].str.strip() == '')]
print(f"\nArticles without a title: {len(no_title)}")
if not no_title.empty:
    print(no_title[['article_url', 'title']])

# Check for articles without text body
no_text = filtered_articles_df[filtered_articles_df['text_body'].isnull() | (filtered_articles_df['text_body'].str.strip() == '')]
print(f"\nArticles without text body: {len(no_text)}")
if not no_text.empty:
    print(no_text[['article_url', 'title']])


Articles with duplicate URLs: 0

Articles without a title: 0

Articles without text body: 9
                                           article_url  \
331  https://www.newsbomb.gr/kosmos/video/1188315/m...   
355  https://www.newsbomb.gr/kosmos/video/1183939/t...   
372  https://www.newsbomb.gr/kosmos/video/1181183/t...   
415  https://www.newsbomb.gr/kosmos/video/1178476/m...   
424  https://www.newsbomb.gr/kosmos/video/1177453/n...   
468  https://www.newsbomb.gr/kosmos/video/1164362/h...   
503  https://www.newsbomb.gr/kosmos/video/1159378/o...   
540  https://www.newsbomb.gr/kosmos/video/1158986/o...   
627  https://www.newsbomb.gr/kosmos/video/1142715/t...   

                                                 title  
331  Μπάιντεν για καταδίκη Σόβιν: Mπορεί να αποτελέ...  
355  Τέξας: Η σπαρακτική κραυγή ενός παιδιού μετανα...  
372  Tραμπ σε γαμήλιο πάρτι στη Φλόριντα: «Σας έλει...  
415  Μπάιντεν: Μπέρδεψε τα πόδια του! Έπεσε τρεις φ...  
424  Ντοναλντ Τραμπ: Συνιστώ να κάνετε το

In [ ]:
biden_articles_20_21= filtered_articles_df

In [ ]:
# Drop specific rows by index
biden_articles_20_21 = biden_articles_20_21.drop([331, 355, 372, 415, 424, 468, 503, 540, 627])

In [ ]:
from google.colab import drive

In [ ]:
scrape_newsbomb_articles_pages(biden_teasers_df).to_csv('/content/drive/MyDrive/biden_articles_20_21.to.csv', index=False)

Processing rows: 100%|██████████| 911/911 [52:07<00:00,  3.43s/it]


Scraping completed. Fetshed 911 articles.
